# Words count with PySpark RDDs
If you've ever heard of "Hello, world!" for web development, "Word count" is the actual equivalent for distributed computing.

In this notebook, we will setup a pipeline that will count the words in a document in a distributed manner. For convenience, we will do this on a single small document, but the operation should easily generalize to bigger documents that would not fit into the memory of a single machine. And that's the idea of this whole module, we work with technology that is able to scale according to the task at hand, even if we practice with smaller data.

In [ ]:
# We start by defining the spark context to play with RDDs
sc = spark.sparkContext

In [ ]:
# We need a S3 filepath

FILENAME = 's3://full-stack-bigdata-datasets/Big_Data/purple_rain.txt'

1. Load the filepath to a Spark RDD using `.textFile(...)` from a SparkContext into `text_file`:

2. Print out `text_file`:

Out[3]: s3://full-stack-bigdata-datasets/Big_Data/purple_rain.txt MapPartitionsRDD[34] at textFile at NativeMethodAccessorImpl.java:0

3. That doesn't tell us much, how would you do to see the first 3 elements of this RDD? take the first 3 elements of the RDD `text_file`:

Out[4]: ['I never meant to cause you any sorrow',
 'I never meant to cause you any pain',
 'I only wanted one time to see you laughing']

4. This is a list of sentences, what we want is a list of tokens. Use the map function and a string method in order to split the charater strings into lists of words:

Out[5]: [['I', 'never', 'meant', 'to', 'cause', 'you', 'any', 'sorrow'],
 ['I', 'never', 'meant', 'to', 'cause', 'you', 'any', 'pain'],
 ['I', 'only', 'wanted', 'one', 'time', 'to', 'see', 'you', 'laughing']]

That's not exactly what we wanted... We wanted a list of tokens, we got a... **list of list of tokens**!

That's because, in this case, we need a special version of `.map()` called `flatMap`: it will flatten the list of list of tokens into a list of tokens.

Let's try it out: we take the same expression as the previous one, but replace `.map()` with `.flatMap()` and call the resulting variable `tokens`.

---
💡 It usually takes time to understand the notion of `.flatMap` and flattening in general, like `.map()`, these are concepts from the functionnal programming world. Unless you come from such a background, it probably **won't be easy to grasp these concepts the first time you encouter them**.

**Let's keep our eyes on the goal: to develop a broader understanding of how Spark works.**

---

5. Copy/paste the previous cell, and:
- replace `.map(...)` with `.flatMap(...)`
- rename the variable `tokenized_text` to `tokens`

6. Use this cell to play with `tokens`, take different amounts of it, or collect it.

Out[7]: ['I', 'never', 'meant', 'to', 'cause', 'you', 'any', 'sorrow', 'I', 'never']

Now that we have our list of words (well, **not exactly a list of words, it is still a RDD**), we can start counting things.

In order to do that, we need to map each word to an initial count, so instead of having:
```
['I',
 'never',
 'meant',
 ...,
 'I',
 'never',
 ...]
```
We would like our list to look like this:
```
[('I', 1),
 ('never', 1),
 ('meant', 1),
 ...,
 ('I', 1),
 ('never', 1),
 ...]
```

7. Write a function `token_to_tuple` that takes:
- a token as input (a string)
- and returns (token, 1) (a tuple) 

8. map `tokens` to your new function `token_to_tuple` to create a variable called `partial_count`

Out[9]: PythonRDD[38] at RDD at PythonRDD.scala:58

9. Take the first 10 elements of `partial_count`

Out[10]: [('I', 1),
 ('never', 1),
 ('meant', 1),
 ('to', 1),
 ('cause', 1),
 ('you', 1),
 ('any', 1),
 ('sorrow', 1),
 ('I', 1),
 ('never', 1)]

Good job!

Beware, now comes the hard-part... We need to reduce this.

Don't forget RDD are very low level objects, when we start using DataFrames (the other main kind of data format) all of this will become easier because of their higher level of abstraction.

What we want is to take tuples with the same key, like `('never', 1)` and `('never', 1)` and aggregate them, so in the end we have `('never', 2)` (or more than 2 if there are more occurences of 'never').

These kind of tuples are called **key-value pairs**, and while most Spark operations work on RDDs containing any type of objects, a few special operations are only available on RDDs of key-value pairs. You can read more about it [in the documentation](https://spark.apache.org/docs/latest/rdd-programming-guide.html#working-with-key-value-pairs).

Among these operations are `.groupByKey(...)` and `.reduceByKey(...)`: the latter has better performances, but the former is easier to understand so we will start with this one.

### groupByKey

10. Call `.groupByKey(...)` on `partial_count` and put it inside `grouped_by_key` variable.

11.  take the first 3 elements of `grouped_by_key`.

Out[12]: [('never', <pyspark.resultiterable.ResultIterable at 0x7fa6180e3430>),
 ('cause', <pyspark.resultiterable.ResultIterable at 0x7fa6180e32b0>),
 ('pain', <pyspark.resultiterable.ResultIterable at 0x7fa6180e32e0>)]

What's this: `<pyspark.resultiterable.ResultIterable at 0x10bc0c2d0>` ?

You don't have to worry about the details, but one thing has to attract your attention: `Iterable`, this seems to suggest those objects are iterable, an iterable in Python is something that you can iterate on: basically something that you can call `for` on, like a list, or a string, etc.

```python
for letter in 'Spark':
    print(letter)
> S
> p
> a
> r
> k
```

Each element of `grouped_by_key` is a tuple, and inside a tuple there is an iterable we can iterate over.

We will first try with the first element.

12. Take the first element of `grouped_by_key`, put it in `first_item` variable and print out its type.

_WARNING: the type should be a tuple, not a list._

Out[18]: ('never', <pyspark.resultiterable.ResultIterable at 0x7fa6118c4340>)

We'd like a way to print these items, for example, such that 'never' would look like this:
```
'never': [1, 1, 1, 1]
```

We will write a function that does this, take an item as a tuple of (`str`, `ResultIterable`), and print out:
```
ITEM_NAME: OCCURENCES_AS_A_LIST
```

13. Define a way to print our items like above:

14. take the first 10 items from `grouped_by_key` and then iterate over them. Then, inside the loop, use the function `print_item(...)` on each item:

never: [1, 1, 1, 1]
cause: [1, 1]
pain: [1]
only: [1, 1, 1, 1, 1, 1, 1]
in: [1, 1]
rain: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Purple: [1, 1, 1, 1, 1, 1, 1, 1, 1]
rain,: [1, 1, 1, 1, 1, 1, 1, 1, 1]
bathing: [1]

Next step might be challenging.

When you take the first 10 elements of `grouped_by_key`, it returns a list of `Tuple[str, ResultIterable]`.  
What we want instead is a list of `Tuple[str, int]` where the second element is the total number of occurence for the fist element.

NOTE: you might wanna try to first return a list of `Tuple[str, list]`.

You should be able to do all this using only list comprehensions.

15. Follow previous instructions:

Out[21]: [('never', 4),
 ('cause', 2),
 ('pain', 1),
 ('only', 7),
 ('in', 2),
 ('rain', 14),
 ('', 10),
 ('Purple', 9),
 ('rain,', 9),
 ('bathing', 1)]

As you've seen this can be done using standard list comprehension.  
If you're curious, even though Python is not a purely functional language, you can write this in a functional fashion and achieve the same result.

_Please note this would look obviouly more elegant in a purely functional language.  
And I put it in there only to introduce you to another programming paradigm if you've mostly encountered imperative programming before. This little introduction is helpful because Spark is based on Scala, which, although not being a purely functional language, provides support for many functional programming features._

In [ ]:
# Don't try at home ;)

from functools import reduce

list(map(lambda t: (t[0], reduce(lambda a, b: a + b, t[1])),
         grouped_by_key.take(10)))

Out[22]: [('never', 4),
 ('cause', 2),
 ('pain', 1),
 ('only', 7),
 ('in', 2),
 ('rain', 14),
 ('', 10),
 ('Purple', 9),
 ('rain,', 9),
 ('bathing', 1)]

That would work, but that's using regular Python, hence we're not profiting from Spark's distributed computing capabilities, which means:

- the computation would be much slower on big datasets,
- if the datasets is too big to be stored on the memory of our machine our program would crash.

That's exactly what `.reduceByKey(...)` will help us to solve. It's usage is a bit similar to `.groupByKey(...)` but it takes a function as a parameter, this function should tell Spark how to aggregate 2 items, in our case, that the value of each tuple. For example, let's say we have a group:

```python
('dog', 1), ('dog', 1)
```

We want a formula applied on values that will give us the end result, e.g. "how many dogs". In our case, that's a simple sum:

```python
def reduce_function(value_1, value_2):
    return value_1 + value_2
```

16. Write our reduce function: reduce_function which takes 2 values and return their sum:

_NOTE: name these parameters `a` and `b`._

17. We're now ready to reduce. You will pass your function as parameter to `.reduceByKey(...)`. call `.reduceByKey(...)` on `partial_count`, name it `reduced`.

18. Take the 10 first values:

Out[25]: [('never', 4),
 ('cause', 2),
 ('pain', 1),
 ('only', 7),
 ('in', 2),
 ('rain', 14),
 ('', 10),
 ('Purple', 9),
 ('rain,', 9),
 ('bathing', 1)]

We're almost there... 😅
We've got a list of tuples, where the key is the token, and the value is its count within the text, but.. 
**they're not ordered...** which is inconvenient if we want to have the 10 most popular tokens within the text.

We will use `.sortBy(...)`, but before we do, let's have a refresher on sorting with Python.

For example, how would you sort this grocery list by the number of items?

In [ ]:
fruits = [('banana', 3), ('orange', 5), ('pineapple', 2)]
fruits

Out[26]: [('banana', 3), ('orange', 5), ('pineapple', 2)]

`sorted(fruits)` won't work because by default sorting on tuple take the first element, in our case, it would sort alphabetically on the name of the fruits.

In [ ]:
sorted(fruits)

Out[27]: [('banana', 3), ('orange', 5), ('pineapple', 2)]

We can force the `key` parameter to sort on the second item of each tuple.

In [ ]:
sorted(fruits, key=lambda x: x[1])

Out[28]: [('pineapple', 2), ('banana', 3), ('orange', 5)]

19. Now, we will do the same on our rdd. Just like `key` in Python's `sorted`, PySpark's `.sortBy(...)` can take a function as a parameter. Use `.sortBy(...)` on `reduced`, name it `sorted_counts`.

20. Take the 10 first values of `sorted_counts`

Out[30]: [('pain', 1),
 ('bathing', 1),
 ('kind', 1),
 ('of', 1),
 ('steal', 1),
 ('end', 1),
 ('underneath', 1),
 ('are', 1),
 ('we', 1),
 ('out', 1)]

What do you think?

It seems sorted, but in **ascending order**!

If we wanted to do this in Python, we could just set the `reverse` argument to `True` when calling `sorted(...)`.

In [ ]:
sorted(fruits, key=lambda x: x[1], reverse=True)

Out[31]: [('orange', 5), ('banana', 3), ('pineapple', 2)]

We can't do this with Spark's RDDs. What we can do instead is **take the opposite value and order by it**.

21. Use `.sortBy(...)` on `reduced`, but with a descending sort, name it `desc_sorted_counts`.

22. Take the 10 first values of `desc_sorted_counts`:

Out[33]: [('rain', 14),
 ('I', 14),
 ('you', 14),
 ('purple', 14),
 ('to', 13),
 ('', 10),
 ('Purple', 9),
 ('rain,', 9),
 ('only', 7),
 ('see', 6)]

Finally, what's the most common word in our document?

### **Bonus**: putting everything together

We will create a function `count_words` that will do everything we did previously, but this time in one swell swoop, we won't use intermediary variables.

The function will:
- take a filepath as argument
- load the content of this filepath into a Spark RDD
- `flatMap(...)` each line of this RDD into tokens by splitting on the ' ' string
- `.map(...)` each token to `(token, 1)` so this can be then reduced
- by calling `.reduceByKey(...)` with a function that sums the values
- and then sort the results with `.sortBy(...)` using the proper function to sort in descending order
- and return an RDD

---
⚠️ Make sure your function returns a RDD

---

In [ ]:
def count_words(filepath):
    # TODO: implement the content of the function
    # 
    # NOTE: you can remove `pass`
    # it's just here to avoid the cell crashing while the
    # content of the function is empty
    pass
    ### BEGIN STRIP ###
    return sc.textFile(filepath)\
    .flatMap(lambda line: line.split(' '))\
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .sortBy(lambda t: -t[1])
    ### END STRIP ###

1. Use `count_words` with `FILENAME` and check its type:

Out[36]: pyspark.rdd.PipelinedRDD

It should be a `pyspark.rdd.PipelineRDD`.

2. Finally, take the 10 first elements of your RDD

Out[37]: [('rain', 14),
 ('I', 14),
 ('you', 14),
 ('purple', 14),
 ('to', 13),
 ('', 10),
 ('Purple', 9),
 ('rain,', 9),
 ('only', 7),
 ('see', 6)]

That's it, you've done it!

You've created a Spark job, the next step would be to neatly package this into a Python executable and submit it to a Spark Cluster for batch or stream execution, but this is beyond the content of this course.

## Going further

We used a toy dataset, we suggest you try with a bigger one.